# Bonus Materials

Vectorised backtesting

[Time Series Momentum](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2089463)

In [ ]:
import pandas as pd

import numpy as np
import pandas as pd
import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import configparser
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

config = configparser.ConfigParser()
config.read('../config/config_v20.ini')
accountID = config['oanda']['account_id']
access_token = config['oanda']['api_key']

client = oandapyV20.API(access_token=access_token)

In [ ]:
params = {
          "count": 5000,
          "granularity": "D"}
r = instruments.InstrumentsCandles(instrument="EUR_USD",
                                   params=params)
res = client.request(r)['candles']

In [ ]:
df = pd.DataFrame([[oo['time'], oo['volume'], float(oo['mid']['o']), 
                    float(oo['mid']['h']), float(oo['mid']['l']), 
                    float(oo['mid']['c'])] for oo in res],
                 columns=['time', 'vol', 'o', 'h', 'l', 'c'])

In [ ]:
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

In [ ]:
df['rtns'] = df['c'].pct_change()

In [ ]:
dtrain = df[0:3000].copy()
dtest = df[3000:].copy()

# Experiment With the Training Data Set

In [ ]:
k = range(50, 300, 20)    # average window length
h = 1     # holding period of 1

In [ ]:
for oo in k:
    dtrain['signal'] = np.sign(dtrain['rtns'].rolling(oo).sum())
    dtrain['strategy_rtn'] =  dtrain['signal'].shift(1) * dtrain['rtns']
    res = dtrain['strategy_rtn'].dropna().sum()
    print('{0:3} {1:>8.4f}'.format(oo, res))

***

# Vectorized Backtesting With the Test Set - Momentum

Note: we use the k=11,h=1 combination as it has the highest returns in the training set 

In [ ]:
dtest['signal'] = np.sign(dtest['rtns'].rolling(90).sum())
dtest['result'] = dtest['signal'].shift(1) * dtest['rtns']
dtest['result'].dropna().cumsum().plot(figsize=(10,6));

***

# Vectorized Backtesting With the Test Set - Reversion

Note: we use the k=3,h=1 combination as it has the highest returns in the training set for a reversion strategy

In [ ]:
dtest['signal'] = - np.sign(dtest['rtns'].rolling(250).sum())
dtest['result'] = dtest['signal'].shift(1) * dtest['rtns']
dtest['result'].dropna().cumsum().plot(figsize=(10,6));